In [ ]:
!pip install lightfm
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808331 sha256=7b0a32b78fa8fdaa99588021d103855efec31005048f036c3c6be2f4da06f943
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=9d2b67b0e1bbc953aefbc1bbb1a5468b98fb6cf4ecc76394488bc4cb5a4bb890
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


# **LightFM Understanding**
LightFM is a Python implementation of a Factorization Machine recommendation algorithm for both implicit and explicit feedbacks

It is a Factorization Machine model which represents users and items as linear combinations of their content features’ latent factors. The model learns embeddings or latent representations of the users and items in such a way that it encodes user preferences over items. These representations produce scores for every item for a given user; items scored highly are more likely to be interesting to the user.

The user and item embeddings are estimated for every feature, and these features are then added together to be the final representations for users and items.

For example, for user i, the model retrieves the i-th row of the feature matrix to find the features with non-zero weights. The embeddings for these features will then be added together to become the user representation e.g. if user 10 has weight 1 in the 5th column of the user feature matrix, and weight 3 in the 20th column, the user 10’s representation is the sum of embedding for the 5th and the 20th features multiplying their corresponding weights. The representation for each items is computed in the same approach.

Modelling approach

Let $U$ be the set of users and $I$ be the set of items, and each user can be described by a set of user features $f_{u} \subset F^{U}$ whilst each items can be described by item features $f_{i} \subset F^{I}$. Both $F^{U}$ and $F^{I}$ are all the features which fully describe all users and items.

The LightFM model operates based binary feedbacks, the ratings will be normalised into two groups. The user-item interaction pairs $(u,i) \in U\times I$ are the union of positive (favourable reviews) $S^+$ and negative interactions (negative reviews) $S^-$ for explicit ratings. For implicit feedbacks, these can be the observed and not observed interactions respectively.

For each user and item feature, their embeddings are $e_{f}^{U}$ and $e_{f}^{I}$ respectively. Furthermore, each feature is also has a scalar bias term ($b_U^f$ for user and $b_I^f$ for item features). The embedding (latent representation) of user $u$ and item $i$ are the sum of its respective features’ latent vectors:

$$
q_{u} = \sum_{j \in f_{u}} e_{j}^{U}
$$

$$
p_{i} = \sum_{j \in f_{i}} e_{j}^{I}
$$

Similarly the biases for user $u$ and item $i$ are the sum of its respective bias vectors. These variables capture the variation in behaviour across users and items:

$$
b_{u} = \sum_{j \in f_{u}} b_{j}^{U}
$$

$$
b_{i} = \sum_{j \in f_{i}} b_{j}^{I}
$$

In LightFM, the representation for each user/item is a linear weighted sum of its feature vectors.

The prediction for user 𝑢 and item 𝑖 can be modeled as the sigmoid of the dot product of user and item vectors, adjusted by their feature biases, as follows:

$$ \hat{r}_{ui} = \sigma(q_u \cdot p_i + b_u + b_i) $$


As the LightFM is constructed to predict binary outcomes e.g. $S^+$ and $S^-$, the function $\sigma()$ is based on the [sigmoid function](https://mathworld.wolfram.com/SigmoidFunction.html).

The LightFM algorithm estimates interaction latent vectors and bias for features. For model fitting, the cost function of the model consists of maximising the likelihood of data conditional on the parameters described above using stochastic gradient descent. The likelihood can be expressed as follows:

$$
L = \prod_{(u,i) \in S+}\hat{r}_{ui} \times \prod_{(u,i) \in S-}1 - \hat{r}_{ui}
$$

Note that if the feature latent vectors are not available, the algorithm will behaves like a [logistic matrix factorisation model](http://stanford.edu/~rezab/nips2014workshop/submits/logmat.pdf).

Reference: https://github.com/recommenders-team/recommenders/blob/main/examples/02_model_collaborative_filtering/lightfm_deep_dive.ipynb

# **LightFM Model Building**


Pre-process Data

In [ ]:
import numpy as np
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, auc_score, recall_at_k
from lightfm.cross_validation import random_train_test_split
import ast
import pickle
from collections import deque
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Googlecolab/user_songs_filtered.csv')
df['toptags'] = df['toptags'].apply(ast.literal_eval)

# Assuming `df` is your DataFrame with the structure provided.
df_copy = df

In [ ]:
df['emotion1'].unique()
emotion_mapping = {
    'joy': 1,
    'positive': 2,
    'trust': 3,
    'surprise': 4,
    'negative': 5,
    'sadness': 6,
    'fear': 7,
    'disgust': 8,
    'anger':9
}
df['emotion1_encoded'] = df['emotion1'].map(emotion_mapping)
df['emotion2_encoded'] = df['emotion2'].map(emotion_mapping)
last_2_interactions = df.groupby('Username').tail(2)
train_df = df.drop(last_2_interactions.index)

Build LightFM Dataset()

In [ ]:
# This is only necessary for colab since it only supports python 3.10, but the library we are using only supports <= 3.9.
# Comment this section if you are running it on your local machine

!sudo rm -rf /usr/local/lib/python3.8/dist-packages/OpenSSL
!sudo rm -rf /usr/local/lib/python3.8/dist-packages/pyOpenSSL-22.1.0.dist-info/

!wget https://repo.anaconda.com/miniconda/Miniconda3-py39_23.5.2-0-Linux-x86_64.sh
!chmod +x Miniconda3-py39_23.5.2-0-Linux-x86_64.sh

!bash ./Miniconda3-py39_23.5.2-0-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.9/site-packages/')
!pip3 install pyOpenSSL==22.0.0

# Installing the recommenders library.
# Ensure that you have python version <=3.9 when installing this.
!pip install recommenders[examples]

--2024-04-16 08:20:17--  https://repo.anaconda.com/miniconda/Miniconda3-py39_23.5.2-0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:20f1, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.191.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93409434 (89M) [application/x-sh]
Saving to: ‘Miniconda3-py39_23.5.2-0-Linux-x86_64.sh’

Miniconda3-py39_23. 100%[===================>]  89.08M   154MB/s    in 0.6s    

2024-04-16 08:20:18 (154 MB/s) - ‘Miniconda3-py39_23.5.2-0-Linux-x86_64.sh’ saved [93409434/93409434]

PREFIX=/usr/local
Unpacking payload ...
                                                                                      
Installing base environment...





Preparing transaction: - \ | / - \ done
Executing transaction: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ 

In [ ]:
from recommenders.evaluation.python_evaluation import precision_at_k, recall_at_k, diversity, map_at_k, ndcg_at_k, auc
from recommenders.utils.timer import Timer
# from recommenders.models.lightfm.lightfm_utils import (
#     track_model_metrics,
#     prepare_test_df,
#     prepare_all_predictions,
#     compare_metric,
#     similar_users,
#     similar_items,
# )
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k,
                                                     recall_at_k, get_top_k_items,
                                                     catalog_coverage, distributional_coverage, novelty, diversity, serendipity)
from recommenders.utils.constants import SEED as DEFAULT_SEED
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import FloatType, IntegerType, LongType, StructType, StructField
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml.feature import HashingTF, CountVectorizer, VectorAssembler
from pyspark.ml.recommendation import ALS
from recommenders.datasets.spark_splitters import spark_random_split
from recommenders.datasets.python_splitters import python_chrono_split, python_stratified_split
from recommenders.evaluation.spark_evaluation import SparkRankingEvaluation, SparkDiversityEvaluation
from recommenders.utils.spark_utils import start_or_get_spark

In [ ]:
dataset = Dataset()
df['track_id'] = df['track_name'] + ' - ' + df['artist_name']
unique_toptags = set(tag for sublist in df['toptags'].dropna() for tag in sublist)
unique_countries = set(df['country'].unique())
item_features_list = ['listeners', 'total_playcount', "profanity_density",
                      "polarity", "subjectivity", "emotion1_encoded", "emotion1_score",
                      "emotion2_encoded", "emotion2_score", "mfcc", "chroma", "rms",
                      "spectral_centroid", "zcr", "tempo"] + list(unique_toptags)
# Preparing the complete list of user features including 'country'
user_features_list = ['registered_year', "track_count", "artist_count"] + list(unique_countries)

#interactions, weights = dataset.build_interactions(((row.Username, row.track_id, row.playcount) for index, row in df.iterrows()))
df['itemID'] = df.groupby('track_id').ngroup() + 1
df['userID'] = df.groupby('Username').ngroup() + 1

df_trial = df.copy()
df_trial.rename(columns={'rank': 'rating'}, inplace=True)
# threshold = 30
# df_trial = df_trial[df_trial.groupby('userID')['userID'].transform('size') >= threshold]
# df_full = df_trial.copy()
df_trial = df_trial[['userID', 'itemID', 'rating']]
df_trial.info()

train, test = python_stratified_split(
    df_trial, ratio=0.8, seed=42
)

# Filtering out users and items in the test set that do not appear in the training set.
# This is done so that we can see if our model has learnt user's previous item interactions and can recommend relevant items.
test = test[test["userID"].isin(train["userID"].unique())]
test = test[test["itemID"].isin(train["itemID"].unique())]

# Creating a test set which only contains the last interaction for each user. Remaining data of the user is used in the train set
leave_one_out_test = test.groupby("userID").last().reset_index()

test.head()

dataset = Dataset()
dataset.fit(
    users=df_trial['userID'].unique(),
    items=df_trial['itemID'].unique(),
    user_features=user_features_list,
    item_features=item_features_list
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393120 entries, 0 to 393119
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   userID  393120 non-null  int64
 1   itemID  393120 non-null  int64
 2   rating  393120 non-null  int64
dtypes: int64(3)
memory usage: 9.0 MB


In [ ]:
df_marked = df.copy()

# Add a marker column in train DataFrame before merging
train['set_type'] = 'train'

# Merge with an indicator to mark the train rows
df_marked = df_marked.merge(train[['userID', 'itemID', 'set_type']], on=['userID', 'itemID'], how='left')

# Rows that didn't match with train will have NaN in set_type, mark these as test
df_marked['set_type'].fillna('test', inplace=True)
df_marked.head()

,Username,country,registered_year,track_count,artist_count,track_name,artist_name,rank,playcount,lyrics,...,rms,spectral_centroid,zcr,tempo,emotion1_encoded,emotion2_encoded,track_id,itemID,userID,set_type
0,emosoup,United States,2015,13520,3386,Higher,Sleep Token,1,1321,you say you wont begin again\ncapitulate and l...,...,0.146034,1508.885765,0.032354,77.133862,5,6,Higher - Sleep Token,53251,4992,train
1,maiconslavieiro,Brazil,2019,18795,6513,Higher,Sleep Token,21,151,you say you wont begin again\ncapitulate and l...,...,0.146034,1508.885765,0.032354,77.133862,5,6,Higher - Sleep Token,53251,6778,train
2,velenious,United States,2013,32367,8810,Higher,Sleep Token,19,1259,you say you wont begin again\ncapitulate and l...,...,0.146034,1508.885765,0.032354,77.133862,5,6,Higher - Sleep Token,53251,9080,train
3,Antimemetic,United States,2023,4030,711,Higher,Sleep Token,32,29,you say you wont begin again\ncapitulate and l...,...,0.146034,1508.885765,0.032354,77.133862,5,6,Higher - Sleep Token,53251,177,train
4,frankcreature,Czech Republic,2023,2528,706,Higher,Sleep Token,43,43,you say you wont begin again\ncapitulate and l...,...,0.146034,1508.885765,0.032354,77.133862,5,6,Higher - Sleep Token,53251,5282,train


In [ ]:
df_train = df_marked[df_marked['set_type']=='train']
df_train['set_type']

0         train
1         train
2         train
3         train
4         train
          ...  
393113    train
393114    train
393116    train
393117    train
393118    train
Name: set_type, Length: 314384, dtype: object

In [ ]:
train_file = "/content/drive/MyDrive/Googlecolab/train.csv"
test_file = "/content/drive/MyDrive/Googlecolab/test.csv"
leave_one_out_test_file = "leave_one_out_test.csv"
train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)


In [ ]:
item_features_data = []
for index, row in df.iterrows():
    chroma_list = np.fromstring(row.chroma.strip('[]'), sep=' ')
    chroma_avg = np.mean(chroma_list)
    mfcc_list = np.fromstring(row.mfcc.strip('[]'), sep=' ')
    mfcc_avg = np.mean(mfcc_list)
    # Preparing a dictionary for the current row/item with feature weights
    features_dict = {
        'listeners': int(row.listeners),
        'total_playcount': int(row.total_playcount),
        'profanity_density': float(row.profanity_density),
        'polarity': float(row.polarity),
        'subjectivity': float(row.subjectivity),
        'emotion1_encoded': int(row.emotion1_encoded),  # Convert to integer if it's encoded as a numeric string
        'emotion1_score': float(row.emotion1_score),
        'emotion2_encoded': int(row.emotion2_encoded),  # Convert to integer if it's encoded as a numeric string
        'emotion2_score': float(row.emotion2_score),
        'mfcc': float(mfcc_avg),
        'chroma': float(chroma_avg),
        'rms': float(row.rms),
        'spectral_centroid': float(row.spectral_centroid),
        'zcr': float(row.zcr),
        'tempo': float(row.tempo)
    }
    toptags_features = {tag: 1.0 for tag in row.toptags}
    features_dict.update(toptags_features)
    # Add the item id and its features to the list
    item_features_data.append((row.itemID, features_dict))

# Now, build the item features matrix with this data
item_features = dataset.build_item_features(item_features_data, normalize=True)
# Save item_features to file
with open('/content/drive/MyDrive/Googlecolab/lightfm_item_features.pkl', 'wb') as file:
    pickle.dump(item_features, file)

In [ ]:
user_features_data = []
# user_features_list = ['country', 'registered_year', "track_count", "artist_count"] + list(unique_countries)
for index, row in df.iterrows():
    # Preparing a dictionary for the current row/item with feature weights
    features_dict = {
        row.country: 1.0,
        'registered_year': int(row.registered_year),
        'track_count': int(row.track_count),
        'artist_count': int(row.artist_count)}
    # Add the item id and its features to the list
    user_features_data.append((row.userID, features_dict))

# Now, build the item features matrix with this data
user_features = dataset.build_user_features(user_features_data, normalize=True)

with open('/content/drive/MyDrive/Googlecolab/lightfm_user_features.pkl', 'wb') as file:
    pickle.dump(user_features, file)


In [ ]:
# Load item_features from file
# import pickle
# with open('/content/drive/MyDrive/Googlecolab/lightfm_item_features.pkl', 'rb') as file:
#     item_features = pickle.load(file)
# # Load user_features from file
# with open('/content/drive/MyDrive/Googlecolab/lightfm_user_features.pkl', 'rb') as file:
#     user_features = pickle.load(file)

train_interactions, train_weights = dataset.build_interactions(
    (row['userID'], row['itemID'], row['rating']) for index, row in train.iterrows()
)

test_interactions, test_weights = dataset.build_interactions(
    (row['userID'], row['itemID'], row['rating']) for index, row in test.iterrows()
)

interactions, weights = dataset.build_interactions(
    (row['userID'], row['itemID'], row['rating']) for index, row in test.iterrows()
)
'''
with open('/content/drive/MyDrive/Googlecolab/interactions.pkl', 'wb') as file:
    pickle.dump(interactions, file)
with open('/content/drive/MyDrive/Googlecolab/weights.pkl', 'wb') as file:
    pickle.dump(weights, file)
'''

"\nwith open('/content/drive/MyDrive/Googlecolab/interactions.pkl', 'wb') as file:\n    pickle.dump(interactions, file)\nwith open('/content/drive/MyDrive/Googlecolab/weights.pkl', 'wb') as file:\n    pickle.dump(weights, file)\n"

## LightFM Model Parameters Explanation

- `SEED = 42`: Sets the seed for pseudorandom number generation to ensure reproducibility of the model training and data splitting.
- `K = 30`: Defines the number of top recommendations to be evaluated (not used in training but often used in evaluation metrics like precision@k).
- `TEST_PERCENTAGE = 0.2`: Specifies that 20% of the data should be set aside as a test dataset to evaluate the model's performance.
- `LEARNING_RATE = 0.05`: Determines the step size at each iteration to minimize the loss function, controlling how quickly the model learns.
- `NO_COMPONENTS = 20`: Number of latent factors to use in the model, representing the dimensions in which users and items are characterized.
- `NO_EPOCHS = 50`: Indicates the number of complete passes through the training dataset the model should make during training.
- `NO_THREADS = 16`: Sets the number of parallel threads to use during model fitting, enhancing computational efficiency on multicore systems.
- `ITEM_ALPHA = 1e-6` and `USER_ALPHA = 1e-6`: Regularization parameters for item and user features to prevent overfitting by penalizing large weights in the model.

### Model Configuration
- The model is configured to use the WARP loss function, which optimizes the order of items in recommendations, focusing on ranking higher the items that should be recommended.

### Model Training
- The model is trained on the interaction data, incorporating user and item features to enhance the recommendations' relevance and accuracy.

### Model Serialization
- Post training, the model is serialized and saved to Google Drive, enabling the persistence of the trained model for future use without the need to retrain.


In [ ]:
# seed for pseudonumber generations
SEED = 42
# default number of recommendations
K = 30
# percentage of data used for testing
TEST_PERCENTAGE = 0.2
# model learning rate
LEARNING_RATE = 0.25
# no of latent factors
NO_COMPONENTS = 20
# no of epochs to fit model
NO_EPOCHS = 50
# no of threads to fit model
NO_THREADS = 32
# regularisation for both user and item features
ITEM_ALPHA = 1e-6
USER_ALPHA = 1e-6
#train_interactions, test_interactions = random_train_test_split(interactions, test_percentage=0.2, random_state=np.random.RandomState(SEED))


Model Training and Saving

In [ ]:
# Train the model
model = LightFM(loss='warp', no_components=NO_COMPONENTS,
                 learning_rate=LEARNING_RATE,
                 random_state=np.random.RandomState(SEED))
model.fit(interactions, user_features=user_features, sample_weight=weights, item_features=item_features, epochs=NO_EPOCHS, num_threads=NO_THREADS)

from datetime import datetime
current_time = datetime.now()
formatted_time = current_time.strftime('%Y-%m-%d_%H:%M:%S')
model_filename = f'/content/drive/MyDrive/Googlecolab/model_lightfm.pkl'  # Update the path to your desired Google Drive folder
import pickle
with open(model_filename, 'wb') as model_file:
    pickle.dump(model, model_file)

Model Evaluation

### Evaluation Metrics Introduction

#### Precision at k:
- **Definition**: Precision at k measures the proportion of recommended items in the top-k set that are relevant.
- **Interpretation**: A high value indicates that many of the top-k recommended items are truly of interest to the user.

#### Recall at k:
- **Definition**: Recall at k assesses the proportion of all relevant items that are captured in the top-k recommendations.
- **Interpretation**: A high value means that the model retrieves a large fraction of the items the user has actually interacted with.

#### AUC Score:
- **Definition**: The Area Under the ROC Curve (AUC) evaluates the model's ability to discriminate between positive and negative interactions.
- **Interpretation**: A score of 1 indicates perfect prediction, while 0.5 indicates no better than random guessing.

#### Reciprocal Rank:
- **Definition**: Reciprocal Rank is the inverse of the rank of the first relevant recommendation.
- **Interpretation**: The higher the value, the more effectively the model places the most relevant item at the top of the recommendation list.


In [ ]:
import pickle
with open('/content/drive/MyDrive/Googlecolab/model_lightfm.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score, reciprocal_rank
# Compute and print the precision at k
precision_at_k = precision_at_k(model, test_interactions, train_interactions= train_interactions,k=K, user_features=user_features, item_features=item_features).mean()
print(f"Precision at k: {precision_at_k}")
recall_at_k = recall_at_k(model, test_interactions, train_interactions= train_interactions,k=K, user_features=user_features, item_features=item_features).mean()
print(f"recall at k: {recall_at_k}")
auc_score = auc_score(model, test_interactions, train_interactions= train_interactions,user_features=user_features, item_features=item_features).mean()
print(f"auc score: {auc_score}")
reciprocal_rank = reciprocal_rank(model, test_interactions, train_interactions= train_interactions, user_features=user_features, item_features=item_features).mean()
print(f"reciprocal_rank: {reciprocal_rank}")

Precision at k: 0.000341358914738521
recall at k: 0.0015155799037485497
auc score: 0.8100261688232422
reciprocal_rank: 0.0018832802306860685


Try out recommending function

In [ ]:
def recommend(user_id, model, data, interactions, n_items=10):
    user_index = data.mapping()[0][user_id]

    scores = model.predict(user_index, np.arange(interactions.shape[1]))

    item_indices = np.argsort(-scores)[:n_items]

    # Convert item indices back to item IDs
    item_ids = [list(data.mapping()[2].keys())[i] for i in item_indices]

    return item_ids

user_id = 3
recommended_tracks = recommend(user_id, model, dataset, interactions, n_items=K)
print(f"Recommended tracks for user {user_id}: {recommended_tracks}")

Recommended tracks for user 3: [23592, 102153, 25103, 66535, 142390, 42598, 101244, 136002, 83027, 27956, 81894, 107228, 137297, 85988, 102147, 40188, 123109, 17921, 87822, 50927, 139560, 95195, 8972, 130238, 61723, 97117, 5133, 56247, 130250, 134043]


In [ ]:
test = pd.read_csv("/content/drive/MyDrive/Googlecolab/test.csv")
test_predictions = [[row.userID, row.itemID, model.predict([row.userID], [row.itemID])[0]]
               for (_, row) in test.iterrows()]
test_predictions = pd.DataFrame(test_predictions, columns=['userID', 'itemID', 'prediction'])
test_predictions.to_csv("/content/drive/MyDrive/Googlecolab/predictions.csv")
test_predictions.head()

,userID,itemID,prediction
0,1,41908,0.000182
1,1,139076,0.000111
2,1,140975,-0.001494
3,1,83514,-0.000900
4,1,27319,-0.001191


In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Googlecolab/train.csv")
train_predictions = [[row.userID, row.itemID, model.predict([row.userID], [row.itemID])[0]]
               for (_, row) in train.iterrows()]

train_predictions = pd.DataFrame(train_predictions, columns=['userID', 'itemID', 'prediction'])
train_predictions.to_csv("/content/drive/MyDrive/Googlecolab/train_predictions.csv")
train_predictions.head()

,userID,itemID,prediction
0,1,83828,-0.004363
1,1,92664,-0.000643
2,1,143073,-0.001625
3,1,18136,0.000184
4,1,115772,0.000492


In [ ]:
all_predictions = pd.concat([train_predictions, test_predictions], ignore_index = True)
all_predictions

,userID,itemID,prediction
0,1,83828,-0.004363
1,1,92664,-0.000643
2,1,143073,-0.001625
3,1,18136,0.000184
4,1,115772,0.000492
...,...,...,...
370944,9483,4959,-591.816467
370945,9483,66874,-591.795349
370946,9483,130206,-591.825867
370947,9483,41908,-591.839600


In [ ]:
TOP_K = 30
eval_map = map_at_k(rating_true=test, rating_pred=all_predictions, col_user='userID', col_item='itemID', col_rating='rating', col_prediction='prediction', k=TOP_K)
print(f"MAP@K: {eval_map}")
eval_ndcg = ndcg_at_k(test, all_predictions, col_user='userID', col_item='itemID', col_rating='rating', col_prediction='prediction', k=TOP_K)
print(f"NDCG@K: {eval_ndcg}")

MAP@K: 0.18761869647521243
NDCG@K: 0.41621593659424766


In [ ]:
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
print(f"Precision: {eval_precision} \n Recall: {eval_recall}")

Precision: 0.15136725133456805 
 Recall: 0.7693460486929274


### Compute Diversity Score


#### Diversity:
- **Definition**: Diversity measures the variety among the recommended items.
- **Interpretation**: High diversity indicates that the model recommends a wide range of distinct items.

#### Novelty:
- **Definition**: Novelty quantifies the unlikeliness of the recommended items based on their overall popularity.
- **Interpretation**: A high novelty score means that the model tends to recommend items that users are less likely to know about.

#### Distributional Coverage:
- **Definition**: Distributional Coverage evaluates the proportion of items in the catalog that are recommended across all users.
- **Interpretation**: High coverage means that the model does not ignore large parts of the catalog in its recommendations.

#### Catalog Coverage:
- **Definition**: Catalog Coverage measures the percentage of the entire catalog represented in the top-k recommendations across users.
- **Interpretation**: Greater catalog coverage indicates a recommendation system that utilizes more of the item space.

#### Serendipity:
- **Definition**: Serendipity measures the element of surprise in the recommendations, considering both their relevance and novelty.
- **Interpretation**: High serendipity values suggest that the model successfully recommends items that are both interesting and not obvious to the user.

In [ ]:
# with Timer() as test_time:

#     users, items, preds = [], [], []
#     item = list(train.itemID.unique())
#     for user in train.userID.unique():

#         user = [user] * len(item)
#         users.extend(user)
#         items.extend(item)
#         preds.extend(list(model.predict(user, item)))

#     all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

#     merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
#     all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

# print("Took {} seconds for prediction.".format(test_time.interval))
# all_predictions = pd.read_csv('/content/drive/MyDrive/all_predictions.csv')


In [ ]:
# Prepare for diversity based evaluations

# Merge all_predictions with test on userID and itemID
merged_df = pd.merge(predictions, test, left_on=['userID', 'itemID'], right_on=['userID', 'itemID'], how='outer')

top_all = merged_df[['userID', 'itemID', 'prediction']]
# print(top_all.shape[0])

# Sort top_all DataFrame by 'prediction' column within each 'userID' group in descending order
top_all_sorted = top_all.sort_values(by=['userID', 'prediction'], ascending=[True, False])

# Group by 'userID' and take the top_k items for each group
top_k_reco = top_all_sorted.groupby('userID').head(K)
print(top_k_reco.shape[0])
#top_k_reco = pd.read_csv('/content/drive/MyDrive/top_k_reco.csv')

56565


In [ ]:
top_k_reco.head()

,userID,itemID,prediction
0,1,41908,0.000182
1,1,139076,0.000111
5,1,15211,-0.000165
6,1,20587,-0.000776
3,1,83514,-0.000900


In [ ]:
eval_diversity = diversity(train, top_k_reco, col_user='userID', col_item='itemID')
print(f"Diversity: {eval_diversity}")

eval_novelty = novelty(train, top_k_reco, col_user='userID', col_item='itemID')
print(f"Novelty: {eval_novelty}")

eval_distributional_coverage = distributional_coverage(train, top_k_reco, col_user='userID', col_item='itemID')
print(f"distributional_coverage: {eval_distributional_coverage}")

eval_catalog_coverage = catalog_coverage(train, top_k_reco, col_user='userID', col_item='itemID')
print(f"catalog_coverage: {eval_catalog_coverage}")

eval_serendipity = serendipity(train, top_k_reco, col_user='userID', col_item='itemID')
print(f"serendipity: {eval_serendipity}")

Diversity: 0.9614247048079936
Novelty: 15.418547336996255
distributional_coverage: 14.087636484456546
catalog_coverage: 0.2067850734707105
serendipity: 0.9732887806474785
